##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimadores preconfigurados

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/premade"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
</table>

> Advertencia: Los estimadores no se recomiendan para código nuevo. Los estimadores ejecutan el código de estilo `v1.Session` que es más difícil de escribir correctamente y que puede tener un comportamiento inesperado, particularmente, cuando se combina con código TF 2. Los estimadores están incluidos dentro de las [garantías de compatibilidad](https://tensorflow.org/guide/versions), pero no se les harán correcciones a menos que se trate de vulneraciones a la seguridad. Para más detalles, consulte la [Guía de migración](https://tensorflow.org/guide/migrate).

En este tutorial se muestra cómo resolver el problema de clasificación de lirios (Iris, en inglés) en TensorFlow con estimadores. Un estimador es una representación de alto nivel clásica de TensorFlow de un modelo completo. Para más detalles, consulte [Estimadores](https://www.tensorflow.org/guide/estimator).

Nota: En TensorFlow 2.0, la [API Keras](https://www.tensorflow.org/guide/keras) puede realizar estas mismas tareas, además, se cree que puede ser una API más fácil de aprender. Si comienza de cero, se recomienda empezar con Keras.


## Primero lo primero

Para empezar, primero importe TensorFlow y las librerías que vaya a necesitar.

In [ ]:
import tensorflow as tf

import pandas as pd

## El conjunto de datos

En el programa de ejemplo que se encuentra en este documento se crea y prueba un modelo en el que se clasifican flores, lirios, en tres especies diferentes según el tamaño de sus [sépalos](https://en.wikipedia.org/wiki/Sepal) y [pétalos](https://en.wikipedia.org/wiki/Petal).

Lo que hará será entrenar un modelo con el conjunto de datos de "lirios". Este conjunto de datos de lirios contiene cuatro atributos y una [etiqueta](https://developers.google.com/machine-learning/glossary/#label). Con los cuatro atributos se identifican las siguientes características botánicas de cada uno de los diferentes lirios:

- largo del sépalo
- ancho del sépalo
- largo del pétalo
- ancho del pétalo

En base a esta información, puede definir algunas constantes útiles para el análisis de los datos:


In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

A continuación, descargue y <em>parsee</em> los datos de los lirios con Keras y Pandas. Tenga en cuenta que se trabajan conjuntos de datos diferentes para entrenamiento y pruebas.

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

Al inspeccionar los datos se ve que hay cuatro columnas de atributos flotantes y una etiqueta int32.

In [ ]:
train.head()

Para cada uno de los conjuntos de datos, separe las etiquetas que el modelo entrenado deberá predecir.

In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

## Descripción general de la programación con estimadores

Ahora que los datos ya están configurados, se puede definir un modelo usando un estimador TensorFlow. Con "un estimador" nos referimos a cualquier clase que derive de `tf.estimator.Estimator`. TensorFlow ofrece una colección de `tf.estimator` (por ejemplo, `LinearRegressor`) para implementar algoritmos de aprendizaje automático comunes. Además de estos, se pueden escribir los propios [estimadores personalizados](https://www.tensorflow.org/guide/estimator#custom_estimators). Se recomienda usar estimadores preconfigurados, simplemente para poder empezar.

Para escribir un programa de TensorFlow basado en estimadores predeterminados, simplemente, debe realizar las siguientes tareas:

- Crear una o más funciones de entrada.
- Definir las columnas de atributos del sistema.
- Instanciar un estimador, especificando las columnas de atributos y varios hiperparámetros.
- Llamar a uno o más métodos sobre el objeto estimador, pasando la función de entrada apropiada como fuente de los datos.

Veamos de qué modo se implementan estas tareas para la clasificación de lirios.

## Creación de funciones de entrada

Se deben crear las funciones de entrada para la evaluación y la predicción, a fin de proporcionar los datos para el entrenamiento.

Una **función de entrada** es una función que devuelve un objeto `tf.data.Dataset` del que salen las siguientes tuplas de dos elementos:

- [`features`](https://developers.google.com/machine-learning/glossary/#feature) - Un diccionario de Python en el cual:
    - Cada clave es el nombre de una función.
    - Cada valor es un arreglo (array) que contiene todos los valores de los atributos.
- `label` - Un arreglo (array) que contiene los valores de la [etiqueta](https://developers.google.com/machine-learning/glossary/#label) para cada uno de los ejemplos.

Con el único objetivo de demostrar el formato de la función de entrada, a continuación, compartimos una implementación simple:

In [ ]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

Con la función de entrada puede generar el diccionario de `features` y la lista de `label` de la forma en que lo desee. Sin embargo, se recomienda usar la [la API de conjunto de datos](https://www.tensorflow.org/guide/datasets) de TensorFlow, que puede <em>parsear</em> todo tipo de datos.

Con la API de dataset se pueden manejar muchos casos comunes. Por ejemplo, con la API de conjuntos de datos (Dataset) se pueden leer fácilmente los registros de una amplia colección de archivos en paralelo y reunirlos en una sola transmisión (o stream).

Para simplificar, en este ejemplo cargará los datos con [pandas](https://pandas.pydata.org/) y creará una canalización de entrada a partir de estos datos en memoria:


In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


## Definición de las columnas de atributos

Una [**columna de atributos**](https://developers.google.com/machine-learning/glossary/#feature_columns) es un objeto que describe de qué modo el modelo debería usar los datos de entrada en bruto del diccionario de atributos. Cuando cree un modelo estimador, pásele una lista de columnas de atributos que describa cada uno de los atributos que desea usar en el modelo. El módulo `tf.feature_column` ofrece muchas opciones para representar los datos del modelo.

Para el caso de los lirios, los 4 atributos en bruto son valores numéricos. Por lo tanto, creará una lista de columnas de atributos para indicarle al modelo estimador que represente cada uno de los cuatro atributos como valores de puntos flotantes de 32-bits. Entonces, el código para crear la columna de atributos es:

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Las columnas de atributos pueden ser mucho más sofisticadas que las que se muestran aquí. Si lo desea, puede leer más sobre las columnas de atributos en [esta guía](https://www.tensorflow.org/guide/feature_columns).

Ahora que tiene la descripción de cómo quiere que el modelo represente los atributos en bruto, puede crear el estimador.

## Instanciación de un estimador

El problema de los lirios es un problema de clasificación clásico. Afortunadamente, TensorFlow ofrece varios estimadores clasificadores preconfigurados, incluidos los siguientes:

- `tf.estimator.DNNClassifier` para modelos profundos con los que se hagan clasificaciones multiclase.
- `tf.estimator.DNNLinearCombinedClassifier` para modelos "wide and deep" (ancho y profundo).
- `tf.estimator.LinearClassifier` para clasificadores basados en modelos lineales.

Para el problema de los lirios, `tf.estimator.DNNClassifier` parece ser la mejor opción. A continuación, mostramos cómo se puede instanciar este estimador:

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

## Entrenamiento, evaluación y predicción

Ahora que ya cuenta con un objeto estimador, puede llamar métodos para que hagan lo siguiente:

- Entrenar el modelo.
- Evaluar el modelo entrenado.
- Usar el modelo entrenado para hacer predicciones.

### Entrenamiento del modelo

Entrene el modelo llamando al método `train` del estimador del siguiente modo:

In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Tenga en cuenta que termina la llamada `input_fn` en una [`lambda`](https://docs.python.org/3/tutorial/controlflow.html) para capturar los argumentos y, a la vez, proporcionar una función de entrada que no use argumentos, tal como lo espera el estimador. El argumento `steps` le dice al método que deje de entrenar después de cierta cantidad de pasos de entrenamiento.


### Evaluación del modelo entrenado.

Ahora que el modelo ya se ha entrenado, es posible obtener algunas estadísticas sobre el desempeño. Con el siguiente bloc de códigos se evalúa la exactitud del modelo entrenado en los datos de prueba:


In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

A diferencia de lo que sucede con la llamada al método `train`, no se pasó el argumento `steps` para evaluar. El código `input_fn` para evaluar solamente produce un solo [ciclo](https://developers.google.com/machine-learning/glossary/#epoch), también conocido como época (epoch), de datos.

El diccionario `eval_result` también contiene la `average_loss` (la pérdida media por muestra), la `loss` (pérdida media por minilote) y el valor del `global_step` del estimador (la cantidad de iteraciones de entrenamiento que se produjeron).


### Predicciones (inferencias) a partir del modelo entrenado

Ya ha entrenado al modelo que produce buenos resultados en la evaluación. Ahora, puede usar el modelo entrenado para predecir la especie de una flor de lirio en base a algunas mediciones sin etiquetar. Del mismo modo que con el entrenamiento y la evaluación, a las predicciones las hará con una llamada a una sola función:

In [ ]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

El método `predict` devuelve un iterable de Python, que produce un diccionario de resultados de predicciones para cada muestra. El siguiente código imprime algunas pocas predicciones y sus probabilidades:

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))